# **Root Diary**

## The Question

This project started with a simple question: how many pages on wikipedia are related to mathematics? 

As it turns out, the answer is not so simple.

## The Problem

Wikipedia's system of categorizing content is all over the place. 

#### _Wikipedia Methods of Categorizing:_

- [Portal page](https://en.wikipedia.org/wiki/Portal:Contents/Portals#Mathematics_and_logic)
- [Contents page](https://en.wikipedia.org/wiki/Portal:Contents/Mathematics_and_logic)
- [Category page](https://en.wikipedia.org/wiki/Category:Mathematics)
- [Outline page](https://en.wikipedia.org/wiki/Outline_of_mathematics)
- [Areas of Mathematics page](https://en.wikipedia.org/wiki/Areas_of_mathematics)
- [Indices](https://en.wikipedia.org/wiki/Category:Mathematics-related_lists)
- [Overviews](https://en.wikipedia.org/wiki/Category:Mathematics-related_lists)
- [Glossaries](https://en.wikipedia.org/wiki/Category:Mathematics-related_lists)

Of these, Mathematics is organized using subset of these structures:

- [Mathematics Portal page](https://en.wikipedia.org/wiki/Portal:Contents/Portals#Mathematics_and_logic)
- [Mathematics Contents page](https://en.wikipedia.org/wiki/Portal:Contents/Mathematics_and_logic)
- [Mathematics Category page](https://en.wikipedia.org/wiki/Category:Mathematics)
- [Mathematics Outline page](https://en.wikipedia.org/wiki/Outline_of_mathematics)
- [Areas of Mathematics page](https://en.wikipedia.org/wiki/Areas_of_mathematics)
- [Mathematics Lists](https://en.wikipedia.org/wiki/Category:Mathematics-related_lists)
- [lists of lists of lists](https://en.wikipedia.org/wiki/List_of_lists_of_lists#Mathematics_and_logic)

This also includes the [list of mathematics topics page](https://en.wikipedia.org/wiki/Lists_of_mathematics_topics)



Contents Overview Outlines Lists Portals Glossaries Categories Indices


#### _Wikidata Methods of Categorizing:_

Outside of the Wikipedia project, the WikiMedia Foundation also includes the the Wikidata project. This project is based on a graph database (using the Sparql language) that is community edited organize data based on their interconnections.

```sparql
 SELECT distinct ?item ?article ?sitelink ?linkTo WHERE {
       { ?item wdt:P361* wd:Q395 .}
       union
       { ?item wdt:P361/wdt:P279* wd:Q395 .}
       union
       { ?item wdt:P31/wdt:P279* wd:Q1936384 .}
       union
       { ?item wdt:P921/wdt:P279* wd:Q395 .}
       optional 
       { ?sitelink ^schema:name ?article .
         ?article schema:about ?item ;
         schema:isPartOf <https://en.wikipedia.org/> .
       }
       OPTIONAL { ?item wdt:P361 ?linkTo. }
       SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
       }
```

While graph databases are dope, when humans are used to create nodes and relationships the data is not entirely reliable.

## Data Wrangling

### _Wikidata Search_

While the results from the Wikidata database search did not answer our original question, we can use the results to create a seed dataset.

While ~800 articles may be enough to train a Natural Language Processing model, more data is always better.

To accomplish this, we can turn to scraping Wikipedia category strucutures. For each category structure page, we can find all the internal wiki links, store them in a container. This represents a depth search of one. Next, scraping the pages in the container for more internal wiki links results in a depth search of two. The number of iterations we perform will be the depth of our search. This can result in a large set of links, however it is a bit crude since collects links  



### _PetScan searching_

Contributors to the Wikimedia Foundation have built a tool for finding articles based on categories. It is a powerful tool, but it is too crude to answer our original question. Instead, we will use it to find our seed datasets. 

PetScan takes in a category parameter and a depth of search parameter. This roughly translates to the same process that we would have

## Traversing Wikipedia

['Wikipedia Data Science: Working with the World’s Largest Encyclopedia'](https://towardsdatascience.com/wikipedia-data-science-working-with-the-worlds-largest-encyclopedia-c08efbac5f5c)

Traversing XML tree iteratively:
xml.sax

parsing wikipedia content pages:
mwparserfromhell